In [1]:
import os
import shutil
from creds import OPENAI_KEY
from application_generator import JobApplicationBuild, JobApplicationBuilder
from utils import LatexToolBox, text_to_pdf
from models import ChatGPT
from prompts.resume_section_prompts import RESUME_WRITER_PERSONA
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

llm = ChatGPT(
    api_key=OPENAI_KEY,
    model="gpt-4o",
    system_prompt=RESUME_WRITER_PERSONA,
    max_output_tokens=None,
    temperature=0.7
)

build = JobApplicationBuild(
    resume_tex_template_path="less_basic_template.tex",
    resume_cls_path="less_basic_template.cls",
    job_content_path=r"input_data/Molina Data Analyst.pdf",
    user_details_content_path=r"input_data/full_resume_contents_20250122.pdf"
)

generator = JobApplicationBuilder(
    llm=llm
)

build = generator.extract_job_content(build)
build = generator.user_data_extraction(build)
build = generator.generate_resume_json(build)
build = generator.resume_json_to_resume_tex(build)
print("Done generating resume tex file")

custom_letter_instructions = input("Enter any custom application instructions for the cover letter: ")
build = generator.generate_cover_letter(build, custom_letter_instructions, need_pdf=False)

# check that the fonts in the resume tex are available and flag the ones not installed
resume_tex_fonts, _ = LatexToolBox.extract_tex_font_dependencies(build.resume_tex_path)
font_statuses = LatexToolBox.check_fonts_installed(resume_tex_fonts)
if not all([v for k, v in font_statuses.items()]):
    [print(f"{k} not installed") for k, v in font_statuses.items() if not v]

generator.cleanup_files(build)


/Users/aaronrdankert/projects/job_hunter_toolbox/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Job Details JSON generated at: output_files/MolinaHealthcare/DataAnalystSqlD_20250219194427/JD.json

Fetching user data...

Generating Resume Details...
Processing Resume's WORK_EXPERIENCE Section...
Processing Resume's PROJECTS Section...
Processing Resume's SKILL_SECTION Section...
Processing Resume's EDUCATION Section...
Processing Resume's CERTIFICATIONS Section...
Processing Resume's ACHIEVEMENTS Section...
Done generating resume tex file
Cover Letter generated at:  output_files/MolinaHealthcare/DataAnalystSqlD_20250219194427/cover_letter.txt


Matplotlib is building the font cache; this may take a moment.


Edit the resume latex and cover letter text before running the next cell.

In [4]:
success = LatexToolBox.compile_latex_to_pdf(
    tex_filepath=build.resume_tex_path, 
    cls_filepath=build.resume_cls_path,
    output_destination_path=build.output_destination
)
if success:
    # Remove auxiliary files using resume base name (without extension)
    base_name = os.path.splitext(os.path.basename(build.resume_tex_path))[0]
    LatexToolBox.cleanup_latex_files(build.output_destination, base_name)
    print(f"Resume PDF is saved at {build.resume_tex_path.replace('.tex','.pdf')}")
else:
    print("LaTeX compilation failed.")

cover_letter_pdf_path = build.cover_letter_path.replace('.txt', '.pdf')
cover_letter_pdf_path = text_to_pdf(build.cover_letter_path, cover_letter_pdf_path)
print(f"Cover Letter PDF is saved at {cover_letter_pdf_path}")

generator.cleanup_files(build)


FileNotFoundError: [Errno 2] No such file or directory: 'less_basic_template.cls'

In [ ]:
# move the job content file to the output folder
shutil.move(job_content_path, generator.get_job_doc_path())

Edit the resume latex and cover letter text before running the next cell.

In [ ]:
LatexToolBox.compile_latex_to_pdf(tex_filepath=resume_tex_path)
print(f"Resume PDF is saved at {resume_tex_path.replace('.tex', '.pdf')}")

cover_letter_pdf_path = cover_letter_txt_path.replace('.txt', '.pdf')
cover_letter_pdf_path = text_to_pdf(cover_letter, cover_letter_pdf_path)
print(f"Cover Letter PDF is saved at {cover_letter_pdf_path}")

KeyboardInterrupt: 

In [ ]:
# move the job content file to the output folder
shutil.move(job_content_path, generator.get_job_doc_path())

In [1]:
import os
import shutil
from creds import OPENAI_KEY
from application_generator import JobApplicationBuilder
from application_generator import JobApplicationBuilder
from utils import LatexToolBox, text_to_pdf
from models import ChatGPT
from prompts.resume_section_prompts import RESUME_WRITER_PERSONA
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

p = r"output_files/MolinaHealthcare/DataAnalystSqlD_20250219194427/resume.tex"
print(os.path.exists(p))

LatexToolBox.compile_latex_to_pdf(tex_filepath=p,
                                  cls_filepath="templates/less_basic_template.cls",
                                  output_destination_path="output_files/MolinaHealthcare/DataAnalystSqlD_20250219194427")

/Users/aaronrdankert/projects/job_hunter_toolbox/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True


True